<h2><center>THE CHINESE UNIVERSITY OF HONG KONG<br /> 
    Department of Mathematics<br />
 MATH4280<br />
    Innovation and Design in Big Data Analytics</center></h2>

# Test 1

## Question 2

In this question, we are going to denoise a color image. 

Enter

In [ ]:
import numpy as np
from scipy.fft import fft2, ifft2, fftshift, ifftshift
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
np.random.seed(1234)
def plot_spectrum(im_fft, ax):
    ax.imshow(np.abs(im_fft), norm=LogNorm(vmin=2))

img = plt.imread('clear_img.png').astype(float)[:,:,:-1]
corrupted_image = plt.imread('corrupted_image.png').astype(float)[:,:,:-1]

Unlike in tutorial, where we dealt with only gray images (2D array), color images are expressed as a 3D array. You are given an image `img` and a corrupted image `corrupted_image`, both of size $(640, 960, 3)$.

More precisely,

The first layer`corrupted_image[:,:,0]` is the value of the red color.

The second layer `corrupted_image[:,:,1]` is the value of the green color.

The third layer `corrupted_image[:,:,2]` is the value of the blue color.

All the values should lie in $[0,1]$.

In [ ]:
fig , ax = plt.subplots(1,2,figsize=(10,10))
print(f"Mean Square Error : {np.mean((corrupted_image-img)**2):f}.")
ax[0].imshow(corrupted_image)
ax[0].set_xticks([]) 
ax[0].set_yticks([])
ax[0].set_title("Corrupted Image")
ax[1].imshow(img)
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_title("Clear Image")
plt.tight_layout()

## Q2a

Create a $2$-by-$3$ plot of the spectrum, using the `plot_spectrum()` function provided.

The first row contains the spectrums of each layer of `corrupted_image` and 
the second row contains those of `img`.

Use `scipy.fft.fft2` and shift the 0-frequency term to the center the of graph.

In [ ]:
## answer of 2a

fig , ax = plt.subplots(2,3,figsize=(10,10))
for i in range(3):
    plot_spectrum(fftshift(fft2(corrupted_image[:,:,i])), ax[0][i])
    ax[0][i].set_xticks([])
    ax[0][i].set_yticks([])
    ax[0][i].set_title(f"Layer {i+1} of Corrupted Image")
for i in range(3):
    plot_spectrum(fftshift(fft2(img[:,:,i])), ax[1][i])
    ax[1][i].set_xticks([])
    ax[1][i].set_yticks([])
    ax[1][i].set_title(f"Layer {i+1} of Clear Image")
plt.tight_layout()

## Q2b 

Plotting the graphs, we can compare the frequencies. We would want to retain the frequencies around the center. 

Select suitable respective filters to retain the frequencies around the center for all the three layers. Save the filtered image as an array `filtered_image`. 

Plot the spectrums after the filtering of these layers.

In [ ]:
## answer of 2b

filtered_image = np.zeros(corrupted_image.shape)
fig, ax = plt.subplots(1,3,figsize=(10,10))
r = [60,100,90]
for i in range(3):
    Y_fft=fftshift(fft2(corrupted_image[:,:,i]))
    rows, cols = Y_fft.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.ones((rows, cols), dtype=bool)
    y, x = np.ogrid[-crow:rows-crow, -ccol:cols-ccol]
    mask[x*x + y*y >= r[i]**2] = 0

    Y_fft_filtered = Y_fft * mask
    filtered_image[:,:,i] = (ifft2(ifftshift(Y_fft_filtered))).real
    plot_spectrum(Y_fft_filtered,ax[i])
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].set_title(f"Layer {i+1}")


plt.tight_layout()

## Q2c 

Reconstruct the image. Plot this next to the original image.

Note that `plt.imshow()` assumes all the values to be in $[0,1]$. You may use `np.clip(filtered_image, 0,1)` to ensure this. 

Calculate the mean square error $= \frac{(\text{filtered\_ image}-\text{img})^2}{\text{size of image}}$. 



In [ ]:
## answer of 2c

fig, ax = plt.subplots(1,2,figsize=(10,10))
ax[0].imshow(np.clip(filtered_image,0,1))
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title("Reconstructed Image")
ax[1].imshow(np.clip(img,0,1))
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_title("Clear Image")
plt.tight_layout()

print(f"The mean square error : {np.mean((filtered_image-img)**2):f}.")

## Q2d

We can see that the reconstructed image is better than the corrupted one but the color tone is off. Why?

## answer of 2d

Because we also remove some of the frequencies of the clear image in 2c.

## Q2e

If we now also have the mean value of each layer of the clear image, how can we use this information to improve our `filtered_image`? 

Save the new result as `new_filtered_image`. Calculate the new mean square error.

In [ ]:
clear_mean = np.zeros(3)
for i in range(3):
    clear_mean[i] = img[:,:,i].mean()

## answer of 2e

new_filtered_image = filtered_image.copy()
for i in range(3):  # For each color channel
    mean_filtered = new_filtered_image[:,:,i].mean()
    new_filtered_image[:,:,i] += clear_mean[i] - mean_filtered


fig, ax = plt.subplots(1,2,figsize=(10,10))
ax[0].imshow(np.clip(new_filtered_image,0,1))
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title("New Reconstructed Image")
ax[1].imshow(np.clip(img,0,1))
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_title("Clear Image")
plt.tight_layout()

print(f"The mean square error : {np.mean((new_filtered_image-img)**2):f}.")